## DATA GENERATION FOR PIXEL-WISE-SUPERVISION SIAMESE NETWORK REPLAY-ATTACK DATASET

### CONVERT VIDEO DATA TO FRAMES

In [2]:
pwd

'/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/Revisiting-Pixel-Wise-Supervision-for-Face-Anti-Spoofing/Pixel Wise Supervision - Siamese-Network'

In [3]:
ls

 DataGeneration.ipynb                              replay-dummy-person-wise/
 dataset/                                          replay-images/
 haarcascade_frontalface_default.xml               test_img.png
'PIxel Wise Supervision - Siamese-Network.ipynb'


In [5]:
import os
import cv2
import dlib
import shutil
import random
import string
import numpy as np
from tqdm.notebook import tqdm
from imutils import face_utils
import matplotlib.pyplot as plt

In [5]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')  

In [6]:
def extract_face(img):
    # Convert to grayscale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
  
    # Detect the faces  
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)  
    
    # Draw rectangle around the faces and crop the faces
    for (x, y, w, h) in faces:
        faces = img[y:y + h, x:x + w]
        
    return faces

In [7]:
# extract_face('./test_img.png')

In [8]:
# START CAPTURING VIDEOS

def extract_multiple_videos(intput_filenames,pathOut,img_class,frame_rate, resize):
    """Extract video files into sequence of images.
       Intput_filenames is a list for video file names"""

    i = 0  # Counter of first video

    # Iterate file names:
    for intput_filename in intput_filenames:
        random_str_count = 7
        img_name = 'client'
        pattern = intput_filename.split('/')
        for p in pattern:
            if 'client' in p:
                f_name = p.split('_')
                for name in f_name:
                    if name.find('client') != -1:
                        img_name = name
        
        # print('the image will be named as:', img_name)
        
        
        sec_count = 0
        cap = cv2.VideoCapture(intput_filename)
        cap.set(cv2.CAP_PROP_POS_MSEC,(sec_count*frame_rate))

        # Keep iterating break
        while True:
#             rand_str = str(''.join(random.choices(string.ascii_uppercase +
#                              string.digits, k = random_str_count)))
            ret, frame = cap.read()  # Read frame from first video

            if ret:
                try:
                    cropped_faces = extract_face(frame)
                    cropped_faces = cv2.resize(cropped_faces,(resize,resize))
                    write_path = pathOut + f'{img_name}_{i}_{img_class}.png'
                    cv2.imwrite(write_path, cropped_faces)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
                    i += 1 # Advance file counter
                    sec_count += 1
                except:
                    pass
            else:
                # Break the interal loop when res status is False.
                break

            # cv2.waitKey(100) #Wait 100msec (for debugging)

        cap.release() #Release must be inside the outer loop
    return i

In [9]:
def start(in_dir , out_dir,img_class,frame_rate=500,resize=256):
    # traverse whole directory
    dir_list = []
    for root, dirs, files in os.walk(f'{in_dir}'):
        # select file name
        for file in files:
            # check the extension of files
            if file.endswith('.mov'):
                # print whole path of files
                path = os.path.join(root, file)
                # print(path)
                dir_list.append(path)
                # extractImages(path,'./data/train/real',frame_rate=15000)
    count = extract_multiple_videos(dir_list,f'{out_dir}',img_class,frame_rate=frame_rate, resize=resize)
    print(f'TOTAL FRAMES/IMAGES FORMED: {count}')

In [10]:
train_real_vids_path = '/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/Revisiting-Pixel-Wise-Supervision-for-Face-Anti-Spoofing/replayattack/train_files/train/real/'
train_spoof_fixed_vids_path = '/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/Revisiting-Pixel-Wise-Supervision-for-Face-Anti-Spoofing/replayattack/train_files/train/attack/fixed/'
train_spoof_hand_vids_path = '/home/pineapple45/ANMOL/BTP/pixel-wise-supervision/Revisiting-Pixel-Wise-Supervision-for-Face-Anti-Spoofing/replayattack/train_files/train/attack/hand/'

In [11]:
train_real_imgs_path = './replay-images/train/real/'
train_spoof_fixed_imgs_path = './replay-images/train/spoof/fixed/'
train_spoof_hand_imgs_path = './replay-images/train/spoof/hand/'

In [12]:
train_dir_real = {"in_dir": train_real_vids_path,"out_dir":train_real_imgs_path}
train_dir_attack_fixed = {"in_dir":train_spoof_fixed_vids_path, 'out_dir':train_spoof_fixed_imgs_path}
train_dir_attack_hand = {"in_dir":train_spoof_hand_vids_path, 'out_dir':train_spoof_hand_imgs_path}

In [13]:
start(train_dir_real['in_dir'],train_dir_real['out_dir'],img_class="real",frame_rate=500,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 22490


In [14]:
start(train_dir_attack_fixed['in_dir'],train_dir_attack_fixed['out_dir'],img_class="attack_fixed",frame_rate=500,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 35466


In [15]:
start(train_dir_attack_hand['in_dir'],train_dir_attack_hand['out_dir'],img_class="attack_hand",frame_rate=500,resize=256) 

TOTAL FRAMES/IMAGES FORMED: 33749


### CHANGE DIRECTORY STRUCTURE BY PLACING IMAGES PERSON WISE

In [1]:
person_wise_dataset_train_path = './dataset/train/images/' 

In [3]:
def get_dir_list(path):
    dir_list = []
    for root, dirs, files in os.walk(f'{path}'):
        # select file name
        for file in files:
            # check the extension of files
            if file.endswith('.png'):
                # print whole path of files
                path = os.path.join(root, file)
                # print(path)
                dir_list.append(path)
    return dir_list

In [33]:
img_path_list_train_real = get_dir_list(train_real_imgs_path)
img_path_list_train_spoof_fixed = get_dir_list(train_spoof_fixed_imgs_path)
img_path_list_train_spoof_hand = get_dir_list(train_spoof_hand_imgs_path)
print(len(img_path_list_train_real)) # ./replay-images/train/real/
print(len(img_path_list_train_spoof_fixed)) # ./replay-images/train/spoof/fixed
print(len(img_path_list_train_spoof_hand)) # ./replay-images/train/spoof/hand

22490
35466
33749


In [34]:
img_path_list_train_real[0]

'./replay-images/train/real/client105_18228_real.png'

In [64]:
def get_client_objects(path_list):
    clients = {}
    for file in path_list:
        path_parts = file.split('/')
        for part in path_parts:
            if 'client' in part:
                file_name_split = part.split('_')
                client_key = file_name_split[0]
                if client_key in clients.keys():
                    clients[client_key].append(file)
                else:
                    clients[client_key] = [file]
    return clients

In [65]:
real_client_objects =  get_client_objects(img_path_list_train_real)
spoof_fixed_client_objects = get_client_objects(img_path_list_train_spoof_fixed)
spoof_hand_client_objects = get_client_objects(img_path_list_train_spoof_hand)

In [77]:
print(sorted(real_client_objects.keys()))
print(sorted(spoof_fixed_client_objects.keys()))
print(sorted(spoof_hand_client_objects.keys()))

['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']
['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']
['client001', 'client002', 'client004', 'client006', 'client007', 'client008', 'client012', 'client016', 'client018', 'client025', 'client027', 'client103', 'client105', 'client108', 'client110']


In [6]:
person_wise_dir_list = get_dir_list(person_wise_dataset_train_path)

In [10]:
ls

 DataGeneration.ipynb                              replay-dummy-person-wise/
 dataset/                                          replay-images/
 haarcascade_frontalface_default.xml               test_img.png
'PIxel Wise Supervision - Siamese-Network.ipynb'


### CREATE list.txt AND APPEND FORMATED PATHS OF IMAGES TO IT

In [11]:
with open('./dataset/list.txt', 'w') as f:
    for path in person_wise_dir_list:
        if '/dataset/train/images/' in path:
            s = path.split('/dataset/train/images/')
            p = s[1]
            f.write("%s\n" % p)

In [88]:
for client_key in sorted(real_client_objects.keys()):
    if os.path.isdir(person_wise_dataset_train_path+client_key):
        shutil.rmtree(person_wise_dataset_train_path+client_key)
    single_client_path = person_wise_dataset_train_path+client_key
    os.mkdir(single_client_path)
    
    single_client_path_real = single_client_path + '/real'
    single_client_path_spoof = single_client_path + '/spoof'
    os.mkdir(single_client_path_real)
    os.mkdir(single_client_path_spoof)
    for raw_client_image_path in real_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_real)
    for raw_client_image_path in spoof_fixed_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_spoof)
    for raw_client_image_path in spoof_hand_client_objects[client_key]:
        shutil.copy2(raw_client_image_path,single_client_path_spoof)